# Importing Required Libraries & Datasets

In [244]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [245]:
sns.set(rc={'figure.figsize':(15,10)})

In [246]:
raw_data = pd.read_csv('loan.csv', index_col='id')

In [247]:
raw_data.head(7)

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
id,,,,,,,,,,,,,,,,,,,,,
312505,312443,500,500,450.0,36 months,9.76%,16.08,B,B2,"Hughes, Hubbard & Reed LLP",...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
242695,242682,500,500,500.0,36 months,10.71%,16.31,B,B5,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
216698,174214,500,500,500.0,36 months,10.46%,16.25,B,B5,THe University of Illinois,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
211133,210644,500,500,475.0,36 months,11.41%,16.47,C,C3,Global Travel International -and- Global Domai...,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
164346,163835,500,500,400.0,36 months,8.07%,15.69,A,A4,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
306018,305720,700,700,700.0,36 months,12.29%,23.35,C,C5,SDSU Residential Life,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
311591,311571,725,725,650.0,36 months,7.37%,22.51,A,A1,"Team Awesome Productions, Inc.",...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


# 1. Cleaning Data

### This step involves cleaning data and converting them into respective formats which can be easily considered for quick calculations

In [248]:
for a in raw_data.columns:
    print(a, '==>', raw_data[a].dtypes)

member_id ==> int64
loan_amnt ==> int64
funded_amnt ==> int64
funded_amnt_inv ==> float64
term ==> object
int_rate ==> object
installment ==> float64
grade ==> object
sub_grade ==> object
emp_title ==> object
emp_length ==> object
home_ownership ==> object
annual_inc ==> float64
verification_status ==> object
issue_d ==> object
loan_status ==> object
pymnt_plan ==> object
url ==> object
desc ==> object
purpose ==> object
title ==> object
zip_code ==> object
addr_state ==> object
dti ==> float64
delinq_2yrs ==> int64
earliest_cr_line ==> object
inq_last_6mths ==> int64
mths_since_last_delinq ==> float64
mths_since_last_record ==> float64
open_acc ==> int64
pub_rec ==> int64
revol_bal ==> int64
revol_util ==> object
total_acc ==> int64
initial_list_status ==> object
out_prncp ==> float64
out_prncp_inv ==> float64
total_pymnt ==> float64
total_pymnt_inv ==> float64
total_rec_prncp ==> float64
total_rec_int ==> float64
total_rec_late_fee ==> float64
recoveries ==> float64
collection_recove

### 1.1 Formatting Loan Term Column

In [249]:
def extract_valid_month_from_months_column(row):
    row = re.sub('[A-Za-z, ]', '', str(row))
    return int(row)

In [250]:
raw_data['term'] = raw_data.term.apply(extract_valid_month_from_months_column)

### 1.2 Formatting Interest Column to Float Value

In [251]:
def extract_valid_float_from_interest_column(row):
    row = re.sub('[%, ]', '', str(row))
    return float(row)

In [252]:
raw_data['int_rate'] = raw_data.int_rate.apply(extract_valid_float_from_interest_column)

### 1.3 Formatting Sub-Grade Column

##### While comparing Grade & Sub-Grade column, it is observed that, alphabet in Sub-Grade column is same as alphabet in Grade Column. This can be verified by printing rows where value of "Grade" column is not equal to alphabet in "Sub-Grade" column. (If there are no values i.e. count is zero, while comparing this, we can be assured to drop alphabet from Sub-Grade column)

In [253]:
raw_data[raw_data['grade'] != raw_data.sub_grade.apply(lambda x: x[:1])].grade.count()

0

##### So, we will proceed ahead with dropping the alphabet from Sub-Grade column

In [254]:
def remove_alphabet_from_sub_grade_column(row):
    row = re.sub('[A-Za-z]', '', str(row))
    return int(row)

In [255]:
raw_data['sub_grade'] = raw_data.sub_grade.apply(remove_alphabet_from_sub_grade_column)

### 1.4 Formatting Employee Length (Years of Experience) Column

In [256]:
def clean_employee_length_column(row):
    row = re.sub('[A-Za-z, ]', '', str(row))
    if row == '<1':
        return 0
    elif row == '10+':
        return 10
    elif row == '':
        return np.nan
    else:
        return float(row)

In [257]:
raw_data['emp_length'] = raw_data.emp_length.apply(clean_employee_length_column)

### 1.5 Formatting Ownership Column (Creating new mapped column)

In [258]:
ownership_status_list = list(raw_data.home_ownership.unique())
ownership_status_list.sort()

In [259]:
ownership_dict = {value: index for index, value in enumerate(ownership_status_list)}
ownership_dict

{'MORTGAGE': 0, 'NONE': 1, 'OTHER': 2, 'OWN': 3, 'RENT': 4}

In [260]:
raw_data['home_ownership_mapped'] = raw_data.home_ownership.map(ownership_dict)

### 1.6 Formatting Salary Verification Column (Creating new mapped column)

In [261]:
salary_verify_status_list = list(raw_data.verification_status.unique())
salary_verify_status_list.sort()

In [262]:
income_verification_dict = {value: index for index, value in enumerate(salary_verify_status_list)}
income_verification_dict

{'Not Verified': 0, 'Source Verified': 1, 'Verified': 2}

In [263]:
raw_data['verification_status_mapped'] = raw_data.verification_status.map(income_verification_dict)

### 1.7 Formatting Loan Issue Date to datetime object

In [264]:
raw_data['issue_d'] = pd.to_datetime(raw_data.issue_d, format='%b-%y')

### 1.8 Formatting Loan Status Column (Creating new mapped column)

In [265]:
loan_status_list = list(raw_data.loan_status.unique())
loan_status_list.sort()

In [266]:
loan_status_dict = {value: index for index, value in enumerate(loan_status_list)}
loan_status_dict

{'Charged Off': 0, 'Current': 1, 'Fully Paid': 2}

In [267]:
raw_data['loan_status_mapped'] = raw_data.loan_status.map(loan_status_dict)

### 1.9 Payment Plan Column can be dropped off, as all the records in this column contains same value

In [268]:
if 'pymnt_plan' in raw_data.columns:
    raw_data = raw_data.drop(columns=['pymnt_plan'])

### 1.10 Formatting Purpose Column

In [269]:
raw_data['purpose'] = raw_data.purpose.apply(lambda x: x.replace('_', ' ').title())

### 1.11 Mapping Address State Column With Complete State Name

##### Creating a dictionary of all the states In US and their respective state abbreviation

In [270]:
full_state_list = {'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'}

In [271]:
raw_data['addr_state_name'] = raw_data.addr_state.map(full_state_list)

### 1.12 Formatting Earliest Credit Line Column to Valid Date object

In [272]:
raw_data['earliest_cr_line'] = pd.to_datetime(raw_data.earliest_cr_line, format='%b-%y')

In [273]:
raw_data.earliest_cr_line.describe()

<ipython-input-273-ddce41ebc1d2>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  raw_data.earliest_cr_line.describe()


count                   39717
unique                    526
top       1998-11-01 00:00:00
freq                      370
first     1969-02-01 00:00:00
last      2068-12-01 00:00:00
Name: earliest_cr_line, dtype: object

### 1.13 Formatting Months Since Last Delinq(Default) & Months Since Last Record Column to Valid format

In [274]:
def clean_months_since_last_column(row):
    if type(row) == str:
        return np.nan
    else:
        return float(row)

In [275]:
raw_data['mths_since_last_delinq'] = raw_data.mths_since_last_delinq.apply(clean_months_since_last_column)

In [276]:
raw_data['mths_since_last_record'] = raw_data.mths_since_last_record.apply(clean_months_since_last_column)

### 1.14 Formatting Months Since Last Delinq(Default) & Months Since Last Record Column to Valid format

In [277]:
raw_data['revol_util'] = raw_data.revol_util.apply(extract_valid_float_from_interest_column)

### 1.15 Initial Status Column can be dropped off, as all the records in this column contains same value.

In [278]:
if 'initial_list_status' in raw_data.columns:
    raw_data = raw_data.drop(columns=['initial_list_status'])

In [279]:
raw_data.out_prncp_inv.dtypes

dtype('float64')

### 1.16 Formatting Last Payment, Next Payment & Last Credit Pull Date Column to Valid format

In [280]:
raw_data['last_pymnt_d'] = pd.to_datetime(raw_data.last_pymnt_d, format='%b-%y')

In [281]:
raw_data['next_pymnt_d'] = pd.to_datetime(raw_data.next_pymnt_d, format='%b-%y')

In [282]:
raw_data['last_credit_pull_d'] = pd.to_datetime(raw_data.last_credit_pull_d, format='%b-%y')

### 1.17 Application Type Column can be dropped off, as all the records in this column contains same value.

In [283]:
if 'application_type' in raw_data.columns:
    raw_data = raw_data.drop(columns=['application_type'])

### 1.18 All the columns after Application Type Column are in the correct format (i.e. float value). Hence, those columns need not have to be cleaned

# ==> 2. Comparing Data Between Charged Off & Fully Paid Customers <==

In [284]:
charged_off_df = raw_data[raw_data['loan_status'] == 'Charged Off']

In [285]:
fully_paid_df = raw_data[raw_data['loan_status'] == 'Fully Paid']

### 2.1 Comparing if "Term" Column Impacts Defaulter / Paid Status

In [298]:
# charged_off = charged_off_df.groupby(by='term').verification_status.count() / len(charged_off_df) * 100
# charged_off = charged_off.rename("Charged Off")
# fully_paid = fully_paid_df.groupby(by='term').verification_status.count() / len(fully_paid_df) * 100
# fully_paid = fully_paid.rename("Fully Paid")
# pd.concat([charged_off, fully_paid], axis=1, join='outer')
raw_data.groupby(by=['loan_status', 'term']).term.count()

loan_status  term
Charged Off  36       3227
             60       2400
Current      60       1140
Fully Paid   36      25869
             60       7081
Name: term, dtype: int64

### 2.3 Comparing if "Verification Status" Column Impacts Defaulter / Paid Status

In [293]:
# charged_off = charged_off_df.groupby(by='verification_status').verification_status.count() / len(charged_off_df) * 100
# charged_off = charged_off.rename("Charged Off")
# fully_paid = fully_paid_df.groupby(by='verification_status').verification_status.count() / len(fully_paid_df) * 100
# fully_paid = fully_paid.rename("Fully Paid")
# pd.concat([charged_off, fully_paid], axis=1, join='outer')
raw_data.groupby(by=['loan_status', 'verification_status']).verification_status.count()

loan_status  verification_status
Charged Off  Not Verified            2142
             Source Verified         1434
             Verified                2051
Current      Not Verified             227
             Source Verified          310
             Verified                 603
Fully Paid   Not Verified           14552
             Source Verified         8243
             Verified               10155
Name: verification_status, dtype: int64

##### Note: Values are almost the same for Charged Off & Fully Paid Customers. So it does not provide great insight

### 2.4 Comparing if "Purpose" Column Impacts Defaulter / Paid Status

In [299]:
# charged_off = charged_off_df.groupby(by='purpose').purpose.count().sort_values(ascending=False) / len(charged_off_df) * 100
# charged_off = charged_off.rename("Charged Off")
# fully_paid = fully_paid_df.groupby(by='purpose').purpose.count().sort_values(ascending=False) / len(fully_paid_df) * 100
# fully_paid = fully_paid.rename("Fully Paid")
# pd.concat([charged_off, fully_paid], axis=1, join='outer')
raw_data.groupby(by=['loan_status', 'purpose']).purpose.count()

loan_status  purpose           
Charged Off  Car                     160
             Credit Card             542
             Debt Consolidation     2767
             Educational              56
             Home Improvement        347
             House                    59
             Major Purchase          222
             Medical                 106
             Moving                   92
             Other                   633
             Renewable Energy         19
             Small Business          475
             Vacation                 53
             Wedding                  96
Current      Car                      50
             Credit Card             103
             Debt Consolidation      586
             Home Improvement        101
             House                    14
             Major Purchase           37
             Medical                  12
             Moving                    7
             Other                   128
             Renewable En

##### Note: Values are almost the same for Charged Off & Fully Paid Customers. So it does not provide great insight

### 2.4 Comparing if "Home Ownership" Column Impacts Defaulter / Paid Status

In [300]:
# charged_off = charged_off_df.groupby(by='home_ownership').home_ownership.count().sort_values(ascending=False) / len(charged_off_df) * 100
# charged_off = charged_off.rename("Charged Off")
# fully_paid = fully_paid_df.groupby(by='home_ownership').home_ownership.count().sort_values(ascending=False) / len(fully_paid_df) * 100
# fully_paid = fully_paid.rename("Fully Paid")
# pd.concat([charged_off, fully_paid], axis=1, join='outer')
raw_data.groupby(by=['loan_status', 'home_ownership']).home_ownership.count()

loan_status  home_ownership
Charged Off  MORTGAGE           2327
             OTHER                18
             OWN                 443
             RENT               2839
Current      MORTGAGE            638
             OWN                  83
             RENT                419
Fully Paid   MORTGAGE          14694
             NONE                  3
             OTHER                80
             OWN                2532
             RENT              15641
Name: home_ownership, dtype: int64

##### Note: Values are almost the same for Charged Off & Fully Paid Customers. So it does not provide great insight

### 2.4 Comparing if "DTI" Column Impacts Defaulter / Paid Status

In [301]:
charged_off_df.dti.describe()

count    5627.000000
mean       14.000624
std         6.585577
min         0.000000
25%         9.050000
50%        14.290000
75%        19.290000
max        29.850000
Name: dti, dtype: float64

In [302]:
fully_paid_df.dti.describe()

count    32950.000000
mean        13.148421
std          6.680041
min          0.000000
25%          7.980000
50%         13.200000
75%         18.390000
max         29.990000
Name: dti, dtype: float64